<a href="https://colab.research.google.com/github/RaziehAkbari2020/Search_Tavily-with-Langchain/blob/main/Tavily%2C_LLamaindex_and_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install transformers
!pip install tavily
!pip install llama-index


ERROR: Could not find a version that satisfies the requirement tavily (from versions: none)
ERROR: No matching distribution found for tavily
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.4/328.4 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.8 MB/s eta 0:00:00


In [8]:
!pip install --upgrade llama-index


In [11]:
!pip install llama-index-embeddings-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.2/853.2 kB 10.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-many

In [14]:

from transformers import RagTokenizer, RagRetriever, RagTokenForGeneration
from tavily import TavilyClient
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import Settings

# Setup logging
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# Initialize Tavily client
tavily = TavilyClient(api_key="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")

# Perform advanced search
response = tavily.search(query="I want to apply for Search Master's degree in Applied Health Informatics & Digital Medicine at RWTH Aachen University in Germany, the deadline, Language requirements, the grade of language, GPA requirement?", search_depth="advanced")

# Extracting URL and content from the response
context = [{"url": obj["url"], "content": obj["content"]} for obj in response["results"]]

# Save the context to a text file (SimpleDirectoryReader requires documents to be saved in files)
with open('context.txt', 'w') as f:
    for item in context:
        f.write(f"URL: {item['url']}\nContent: {item['content']}\n\n")

# Load documents with SimpleDirectoryReader
documents = SimpleDirectoryReader('path_to_your_directory').load_data()

# Create an index with VectorStoreIndex
index = VectorStoreIndex.from_documents(documents)

# Initialize RAG components
tokenizer = RagTokenizer.from_pretrained("facebook/rag-token-nq")
retriever = RagRetriever.from_pretrained("facebook/rag-token-nq", index=index)
model = RagTokenForGeneration.from_pretrained("facebook/rag-token-nq")

# Create a question based on the context
question = "What are the deadlines, language requirements, and GPA requirements for applying for a Master's degree in Applied Health Informatics & Digital Medicine at RWTH Aachen University?"

# Tokenize the question
input_ids = tokenizer(question, return_tensors="pt").input_ids

# Generate response using RAG
outputs = model.generate(input_ids, num_beams=4, num_return_sequences=1)

# Decode and print the response
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(response[0])


ModuleNotFoundError: No module named 'llama_index.llms.huggingface'